# Download data from Kaggle

In [ ]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c human-activity-recognition-har
!unzip /content/human-activity-recognition-har.zip

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          13025        267  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51           9900        210  1.0              
dataanalyst001/population-of-all-us-cities-2024                     Population of all US Cities 2024                      8KB  2024-07-04 12:00:17            742         22  1.0              
ihelon/coffee-sales                     

In [ ]:
import os
from glob import glob
from natsort import os_sorted
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
input_dir = "/content/HAR"
train_dir = input_dir + "/train"
test_dir  = input_dir + "/test"
df_paths  = glob(os.path.join(train_dir,'*','*.csv'))

# Combine df

In [ ]:
concatdf = []
for activity_dir in os.listdir(train_dir):
    if activity_dir[0] == '.':
        continue
    print(activity_dir)
    for csv_file in os.listdir(os.path.join(train_dir,activity_dir)):
        print(os.path.join(train_dir, activity_dir, csv_file))
        df = pd.read_csv(os.path.join(train_dir, activity_dir, csv_file))
        df['label'] = activity_dir
        print(csv_file,df.shape[0],"->",200*(df.shape[0]//200))
        concatdf.append(df[:200*(df.shape[0]//200)])
        # break
    # break
frame = pd.concat(concatdf)

Downstairs
/content/HAR/train/Downstairs/13.csv
13.csv 4241 -> 4200
/content/HAR/train/Downstairs/12.csv
12.csv 2870 -> 2800
/content/HAR/train/Downstairs/8.csv
8.csv 3346 -> 3200
/content/HAR/train/Downstairs/15.csv
15.csv 1762 -> 1600
/content/HAR/train/Downstairs/3.csv
3.csv 3326 -> 3200
/content/HAR/train/Downstairs/14.csv
14.csv 2875 -> 2800
/content/HAR/train/Downstairs/27.csv
27.csv 3460 -> 3400
/content/HAR/train/Downstairs/1.csv
1.csv 2941 -> 2800
/content/HAR/train/Downstairs/20.csv
20.csv 4673 -> 4600
/content/HAR/train/Downstairs/10.csv
10.csv 3795 -> 3600
/content/HAR/train/Downstairs/24.csv
24.csv 2929 -> 2800
/content/HAR/train/Downstairs/16.csv
16.csv 1575 -> 1400
/content/HAR/train/Downstairs/17.csv
17.csv 3767 -> 3600
/content/HAR/train/Downstairs/30.csv
30.csv 3872 -> 3800
/content/HAR/train/Downstairs/28.csv
28.csv 2997 -> 2800
/content/HAR/train/Downstairs/19.csv
19.csv 2614 -> 2600
/content/HAR/train/Downstairs/18.csv
18.csv 2415 -> 2400
/content/HAR/train/Downsta

In [ ]:
frame

,x-axis,y-axis,z-axis,label
0,2.87,6.36,1.988571,Downstairs
1,0.46,5.13,2.492524,Downstairs
2,0.65,4.10,2.451662,Downstairs
3,1.12,3.87,3.023717,Downstairs
4,-3.83,10.38,0.926184,Downstairs
...,...,...,...,...
3595,-3.83,6.51,1.990000,Upstairs
3596,-0.84,9.53,2.830000,Upstairs
3597,-2.98,13.33,-0.460000,Upstairs
3598,-1.88,12.30,0.890000,Upstairs


# Apply mean and std

In [ ]:
nr = 200
new = []
frame = frame
for i in tqdm(range(len(frame)//nr)):
    x = frame[i*nr:(i+1)*nr]
    a = x.groupby(['label'])[['x-axis','y-axis','z-axis']].mean()
    a = a.rename(columns={'x-axis':'x-axismean','y-axis':'y-axismean','z-axis':'z-axismean'})
    b = x.groupby(['label'])[['x-axis','y-axis','z-axis']].std()
    b = b.rename(columns={'x-axis':'x-axisstd','y-axis':'y-axisstd','z-axis':'z-axisstd'})
    ab = a.join(b)
    new.append(ab)
train_df = pd.concat(new)
train_df

100%|██████████| 4345/4345 [00:47<00:00, 90.81it/s] 


,x-axismean,y-axismean,z-axismean,x-axisstd,y-axisstd,z-axisstd
label,,,,,,
Downstairs,-1.74280,9.48620,1.890436,2.512704,4.015490,2.360085
Downstairs,-2.30950,9.46585,1.811711,2.383877,3.778605,2.641734
Downstairs,-2.59120,9.49480,1.489862,2.627285,3.842479,2.757268
Downstairs,-2.34880,9.55530,1.257839,2.345217,3.702459,2.587470
Downstairs,-1.69530,9.51495,1.806943,2.043725,2.905631,2.091827
...,...,...,...,...,...,...
Upstairs,-3.75910,7.81220,-1.556750,5.995069,4.794219,3.052279
Upstairs,-7.07440,7.83220,-0.587100,3.756182,4.779653,3.297324
Upstairs,-7.16725,7.83830,-0.762300,3.336960,4.515456,3.830489


In [ ]:
# Create dummy axis
train_df['activity'] = train_df.index
# Remove axis
train_df.reset_index(drop=True, inplace=True)

# Train Model

In [ ]:
!pip install catboost
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
X = train_df.drop(columns=['activity'])
y = train_df['activity']

## lightGBM

In [ ]:
import lightgbm as lgb

In [ ]:

LightGMB_model = lgb.LGBMClassifier()
LightGMB_model.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 4345, number of used features: 6
[LightGBM] [Info] Start training from score -2.439245
[LightGBM] [Info] Start training from score -1.158604
[LightGBM] [Info] Start training from score -2.996884
[LightGBM] [Info] Start training from score -3.189395
[LightGBM] [Info] Start training from score -2.200914
[LightGBM] [Info] Start training from score -0.923799
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

LGBMClassifier()

## Catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
Catboost_model = CatBoostClassifier()
Catboost_model.fit(X, y)

Learning rate set to 0.085283
0:	learn: 1.5702800	total: 111ms	remaining: 1m 51s
1:	learn: 1.3983373	total: 153ms	remaining: 1m 16s
2:	learn: 1.2782921	total: 202ms	remaining: 1m 7s
3:	learn: 1.1812284	total: 252ms	remaining: 1m 2s
4:	learn: 1.1011049	total: 317ms	remaining: 1m 3s
5:	learn: 1.0330600	total: 346ms	remaining: 57.4s
6:	learn: 0.9751741	total: 395ms	remaining: 56s
7:	learn: 0.9227037	total: 422ms	remaining: 52.3s
8:	learn: 0.8773375	total: 453ms	remaining: 49.9s
9:	learn: 0.8376852	total: 486ms	remaining: 48.1s
10:	learn: 0.8035130	total: 532ms	remaining: 47.8s
11:	learn: 0.7719523	total: 599ms	remaining: 49.3s
12:	learn: 0.7429710	total: 639ms	remaining: 48.6s
13:	learn: 0.7170109	total: 681ms	remaining: 47.9s
14:	learn: 0.6914389	total: 714ms	remaining: 46.9s
15:	learn: 0.6691821	total: 743ms	remaining: 45.7s
16:	learn: 0.6500076	total: 773ms	remaining: 44.7s
17:	learn: 0.6317615	total: 807ms	remaining: 44s
18:	learn: 0.6153824	total: 844ms	remaining: 43.6s
19:	learn: 0.

## XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Convert categorical labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

XGBoost_model = XGBClassifier()
XGBoost_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

<hr>

# ensemble model

In [ ]:
submit_df = pd.read_csv("/content/sample_submission.csv")
folder_path = "/content/HAR/test"
for idx,row in submit_df[:].iterrows():
    csvfile = os.path.join(folder_path,row['id'])
    x = pd.read_csv(csvfile)
    x['label'] = 'xxx'
    a = x.groupby(['label'])[['x-axis','y-axis','z-axis']].mean()
    a = a.rename(columns={'x-axis':'x-axismean','y-axis':'y-axismean','z-axis':'z-axismean'})
    b = x.groupby(['label'])[['x-axis','y-axis','z-axis']].std()
    b = b.rename(columns={'x-axis':'x-axisstd','y-axis':'y-axisstd','z-axis':'z-axisstd'})
    ab = a.join(b)
    ab.reset_index(drop=True, inplace=True)

    lgb_pred = LightGMB_model.predict_proba(ab)
    cat_pred = Catboost_model.predict_proba(ab)
    xgb_pred = XGBoost_model.predict_proba(ab)

    # Average the predictions
    avg_pred = (lgb_pred + cat_pred + xgb_pred) / 3
    result = np.argmax(avg_pred, axis=1)
    # Convert numerical predictions back to original categorical labels
    result = label_encoder.inverse_transform(result)
    submit_df.at[idx,'class'] = result[0]
    # submit_df.at[idx,'class'] = label_encoder.inverse_transform(result)

In [ ]:
import time
version = 'ensemble'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
submit_df.to_csv(csv_name,index=False)

v.ensemble.2020.csv
